In [1]:
from algorithms.persistance import load_from_json

import os
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

In [2]:
def convert_to_df(path: str):
    results = load_from_json(path)
    config_list = []
    performance_list = []
    for result in results:
        config_list.append(result['config'])
        performance_list.append(result['performance'])
    config = pd.DataFrame(config_list)
    performance = pd.DataFrame(performance_list)
    result_list = pd.concat([performance, config], axis=1)
    return result_list

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
name = 'stide_mode_flag_v2'
path = f'../persistent_data/{name}.json'
result_list = convert_to_df(path)
result_list

,true_positives,false_positives,true_negatives,false_negatives,correct_alarm_count,exploit_count,detection_rate,consecutive_false_positives_normal,consecutive_false_positives_exploits,recall,precision_with_cfa,precision_with_syscalls,f1_cfa,scenario,ngram,window,algorithm,detection_time
0,18124,375,1230587,176757,172,174,0.988506,4,0,0.988506,0.977273,0.314442,0.982857,CVE-2017-7529/,3,100,stide,1.550473
1,62580,1223,1229739,132301,172,174,0.988506,4,0,0.988506,0.977273,0.123297,0.982857,CVE-2017-7529/,3,1000,stide,1.558355
2,17954,387,1228761,176927,172,174,0.988506,5,0,0.988506,0.971751,0.307692,0.980057,CVE-2017-7529/,5,100,stide,1.653479
3,62238,1235,1227913,132643,172,174,0.988506,5,0,0.988506,0.971751,0.122246,0.980057,CVE-2017-7529/,5,1000,stide,1.633279
4,17612,380,1226954,177269,172,174,0.988506,5,0,0.988506,0.971751,0.311594,0.980057,CVE-2017-7529/,7,100,stide,1.767648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,69196,1473,174089510,19581335,90,98,0.918367,6,0,0.918367,0.937500,0.057582,0.927835,ZipSlip/,3,300,stide,180.095566
137,149620,3982,174087001,19500911,90,98,0.918367,8,0,0.918367,0.918367,0.022102,0.918367,ZipSlip/,3,700,stide,181.436433
138,110234,2673,174088310,19540297,90,98,0.918367,6,0,0.918367,0.937500,0.032573,0.927835,ZipSlip/,3,500,stide,181.711183
139,46574,873,174090110,19603957,90,98,0.918367,6,0,0.918367,0.937500,0.093458,0.927835,ZipSlip/,3,200,stide,186.180382


In [5]:
conn = sql.connect('stide_mode_flag_v2.db')
try:
    result_list.to_sql(name, conn)
except ValueError:
    print('Table already exists')

In [6]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res.fetchall():
    print(name[0])
name = name[0]

stide_mode_flag_v2


In [7]:
result = pd.read_sql(f'SELECT ngram, \
                              window, \
                              AVG(detection_rate) as DR, \
                              AVG(consecutive_false_positives_exploits)+ AVG(consecutive_false_positives_normal) as CFP_sum, \
                              Count(*), \
                              detection_time as time_in_min \
                              FROM  {name} \
                     GROUP BY ngram, window \
                     ORDER BY DR desc, \
                              CFP_sum asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,ngram,window,DR,CFP_sum,Count(*),time_in_min
0,3,800,0.887093,7.636364,11,1.554296
1,3,900,0.880905,7.700000,10,1.526396
2,3,1000,0.880905,8.200000,10,1.558355
3,3,700,0.874618,7.300000,10,1.525578
4,3,500,0.826585,7.700000,10,1.567505
5,3,600,0.804295,7.000000,10,1.563617
6,3,400,0.790747,8.200000,10,1.548473
7,3,300,0.785585,7.600000,10,1.578467
8,3,200,0.751537,10.900000,10,1.529172
9,3,100,0.693332,11.400000,10,1.550473
